In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras.callbacks import EarlyStopping, CSVLogger, LearningRateScheduler, ModelCheckpoint
from keras.layers import Dense, Flatten, Input, Dropout, BatchNormalization
from keras.models import load_model, Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
path='/content/drive/MyDrive/캡스톤디자인I/EEG_DATA_Shuffle_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF[dataDF.iloc[:, -1] != 2]
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,7.7005,16.5688,14.6191,-2.7372,9.7376,9.6343,12.7423,15.5180,23.1480,3.2137,...,20.2932,19.9336,26.9401,0.4877,15.9825,15.2125,28.8786,13.1696,29.3290,1.0
3,-76.7901,10.4661,60.9218,39.1930,33.3773,65.0877,49.0782,5.1342,43.5867,0.9796,...,-38.8314,-50.8898,-42.2924,-22.9204,-48.7766,-44.7011,-38.4557,-40.1042,-36.8672,1.0
4,9.5458,8.5632,2.2690,-7.9035,7.0756,7.6126,9.5961,8.0069,60.6086,-3.4619,...,-17.7538,-3.9169,-11.5917,-7.9538,-12.3448,-18.7311,-4.8867,-10.0855,5.3741,1.0
5,54.1066,-29.2396,-52.1869,-46.1536,-39.8975,-55.1390,-20.3230,20.2025,-16.8425,16.3951,...,-58.7427,-43.0384,-24.9481,-13.5026,-74.4552,-17.3629,-48.7126,-28.2805,-16.2469,0.0
6,-21.9437,-34.9235,-27.5263,-20.9710,-0.9563,3.7334,-7.1844,-14.7087,3.9823,-16.8421,...,0.6112,-1.8943,-4.1084,6.3730,-5.9389,8.8530,-0.3462,-14.9738,-15.7967,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,31.8298,28.2079,16.0677,16.1556,17.7885,10.1546,5.3282,8.8298,-6.3858,-37.7427,...,-13.2528,-33.0454,-21.1263,-9.9571,-86.3534,-24.2241,-32.5561,-5.7018,3.6393,1.0
3371,-7.0424,-19.2560,1.3570,-6.2609,-12.3790,-10.1178,-10.6833,2.2586,3.8605,-12.5384,...,1.5050,-16.4925,-26.9612,-13.1342,-21.2572,-1.3507,-27.7396,-6.2248,-9.7443,1.0
3372,-34.5889,-16.0601,3.5147,56.5993,-13.7713,-1.4305,18.7316,-2.2079,3.7468,0.6465,...,2.6975,20.9829,10.2222,4.9892,-9.2410,3.5549,13.9819,-3.4836,-1.3360,0.0
3373,-7.7020,-32.1352,-19.0261,18.9835,-12.2132,-22.5264,-5.8123,-6.4590,-868.1081,0.2058,...,-1.5312,-23.7726,-16.1596,3.1612,0.3230,6.0892,-14.2523,3.1717,-37.0443,4.0


In [9]:
ct = 500

x_data = []
y_data = []

print(len(dataDF))

for i in range(len(dataDF)):
    # for i in range(100):
    print(i)
    x_data.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_data.append(one_row_data[y - 1])
    for j in range(ct):
        x_data[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_data = np.array(x_data).reshape(len(x_data), ct, 19)
y_data = np.array(pd.get_dummies(y_data))

x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data, y_data, test_size=0.2, random_state=2024)

x_data_train, x_data_val, y_data_train, y_data_val = train_test_split(x_data_train, y_data_train, test_size=0.25, random_state=2024)

2700
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [10]:
print("Training set shapes:")
print(x_data_train.shape)
print(y_data_train.shape)
print("-------------------")
print("Validation set shapes:")
print(x_data_val.shape)
print(y_data_val.shape)
print("-------------------")
print("Test set shapes:")
print(x_data_test.shape)
print(y_data_test.shape)

Training set shapes:
(1620, 500, 19)
(1620, 4)
-------------------
Validation set shapes:
(540, 500, 19)
(540, 4)
-------------------
Test set shapes:
(540, 500, 19)
(540, 4)


In [11]:
# ct=500

# x_data = []
# y_data = []

# print(len(dataDF))

# for i in range(len(dataDF)):
# # for i in range(100):
#     print(i)
#     x_data.append([])
#     x = 0
#     y = 20
#     one_row_data = dataDF.iloc[i]
#     y_data.append(one_row_data[y-1])
#     for j in range(ct):
#         x_data[i].append([one_row_data[x:y-1]])
#         x += 20
#         y += 20

# x_data = np.array(x_data).reshape(len(x_data), ct, 19)
# y_data = np.array(pd.get_dummies(y_data))

# x_data, x_test_data, y_data, y_test_data = train_test_split(x_data, y_data, test_size=0.2, random_state=2024)

# print('x_data :', x_data.shape)
# print('y_data :', y_data.shape)
# print('x_test_data :', x_test_data.shape)
# print('y_test_data :', y_test_data.shape)

In [12]:
# 트랜스포머 Block 1개
def transformer_block(input):
  x = keras.layers.MultiHeadAttention(num_heads=16, key_dim=1, dropout=0.1)(input, input, return_attention_scores=False)
  x = x + input
  x = keras.layers.LayerNormalization()(x)
  x2 = keras.layers.Dense(19,activation='relu')(x)
  x2 = x + x2
  x3 = keras.layers.LayerNormalization()(x2)
  return x3

In [13]:
# 위의 transformer_block()을 사용해서 모델을 만드는 함수
def build_model(num_block):
  inputs = keras.Input((ct,19))
  x = inputs
  for _ in range(num_block):
    x = transformer_block(x)
  x = keras.layers.GlobalAveragePooling1D()(x)
  output = keras.layers.Dense(4,activation='softmax')(x)
  return keras.Model(inputs,output)

In [14]:
num_block = 5
myMdl = build_model(num_block)
myMdl.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 500, 19)]            0         []                            
                                                                                                  
 multi_head_attention (Mult  (None, 500, 19)              1283      ['input_1[0][0]',             
 iHeadAttention)                                                     'input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 500, 19)              0         ['multi_head_attention[0][0]',
 Lambda)                                                             'input_1[0][0]']             
                                                                                              

In [15]:
# #에포크마다 모델을 저장하고 loss를 기록한 csv 저장을 위한 코드
# root = '/content/drive/MyDrive/캡스톤디자인I/Save_Model/'
# folder_name = '20240411_1modelsave'
# os.mkdir(root+folder_name)
# csv_logger = CSVLogger(root+folder_name+'/training.csv', separator=",", append=True)
# mcp_save_best = tf.keras.callbacks.ModelCheckpoint(
#     root+folder_name+'/{epoch:02d}-{val_loss:.5f}_best', save_best_only=True,
#     monitor='val_loss', verbose=1, mode='min')
# mcp_save = tf.keras.callbacks.ModelCheckpoint(
#     root+folder_name+'/{epoch:02d}-{val_loss:.5f}', save_best_only=False,
#     monitor='val_loss', verbose=1, mode='min')

In [16]:
# Early_stop = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5, restore_best_weights=True)
# myMdl.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss=['categorical_crossentropy'], metrics=['accuracy'])

In [17]:
# #훈련 코드
# trans_h = myMdl.fit(x_data, y_data, batch_size=64, epochs=500, validation_split=0.2)

In [28]:
adam = Adam(learning_rate=0.001)
#옵티마이저를 Adam 알고리즘을 사용합니다. 학습률을 0.001로 설정(학습률은 가중치를 얼마나 업데이트할지 이를 조정하여 모델의 성능을 조정)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc_loss = ModelCheckpoint('/content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_loss_v4.h5', monitor='val_loss', verbose=1, save_best_only=True)
#훈련중 손실값을 모니터링하여 조기 종료하는 콜백 함수
mc = ModelCheckpoint('/content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_acc_v4.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
#모델의 검증 정확도를 확인하여 가장 좋은 시점의 모델을 저장함.
lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
#학습률을 조정하는 콜백 함수
myMdl.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
#모델의 옵티마이저를 설정, 손실함수, 정확도를 출력
trans_h = myMdl.fit(x_data_train, y_data_train,
                   batch_size=64,
                   validation_data=(x_data_val, y_data_val),
                   epochs=200,
                   callbacks=[es, mc, mc_loss, lr_schedule])

Epoch 1/200
26/26 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.9747
Epoch 1: val_accuracy improved from -inf to 0.94259, saving model to /content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_acc_v4.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: val_loss improved from inf to 0.16577, saving model to /content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_loss_v4.h5
26/26 [==============================] - 31s 839ms/step - loss: 0.0774 - accuracy: 0.9747 - val_loss: 0.1658 - val_accuracy: 0.9426 - lr: 0.0010
Epoch 2/200
26/26 [==============================] - ETA: 0s - loss: 0.0803 - accuracy: 0.9809
Epoch 2: val_accuracy did not improve from 0.94259

Epoch 2: val_loss did not improve from 0.16577
26/26 [==============================] - 21s 822ms/step - loss: 0.0803 - accuracy: 0.9809 - val_loss: 0.1900 - val_accuracy: 0.9278 - lr: 9.0484e-04
Epoch 3/200
26/26 [==============================] - ETA: 0s - loss: 0.0484 - accuracy: 0.9877
Epoch 3: val_accuracy improved from 0.94259 to 0.94630, saving model to /content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_acc_v4.h5

Epoch 3: val_loss did not improve from 0.16577
26/26 [==============================] - 22s 841ms/step - loss: 0.0484 - ac

In [31]:
transformer_best = load_model('/content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_acc_v4.h5')
transformer_accuracy = transformer_best.evaluate(x_data_test, y_data_test, verbose=1)[1]   #테스터 데이터를 사용해 모델 정확도 평가
print("Test Acc. Transformer: {:.3f}%".format(transformer_accuracy * 100))

transformer_best_loss = load_model('/content/drive/MyDrive/캡스톤디자인I/Save_Model/best_transformer_loss_v4.h5')
transformer_loss = transformer_best_loss.evaluate(x_data_test, y_data_test, verbose=1)[0]   #테스터 데이터를 사용해 모델의 손실 평가
print("Test val_Loss. transformer:{:.3f}".format(transformer_loss))

y_pred = np.array(list(map(lambda x: np.argmax(x), transformer_best.predict(x_data_test))))   #테스트 데이터에 대한 예측 수행 및 confusion matrix 출력하여 성능 평가
y_data_test = y_data_test.argmax(axis=1)
cm = confusion_matrix(y_data_test, y_pred)
print(classification_report(y_data_test, y_pred, digits=4)) #digits=4로 소수점 지정

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr

In [27]:
acc = trans_h['accuracy']
val_acc = trans_h['val_accuracy']
loss = trans_h['loss']
val_loss = trans_h['val_loss']
plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('Transformer - Training & Validation Acc.', fontsize=16, weight='bold')
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('transformer - Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.show()

TypeError: 'History' object is not subscriptable

In [ ]:
# Early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4, restore_best_weights=True)
# # filename = os.path.join(model_path, 'checkpoint.h5')
# # checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# model_transformer.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#               loss=['categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
# trans_h = model_transformer.fit(x_data, y_data, batch_size=64, epochs=1000, validation_split=0.2, callbacks=[mcp_save_best, mcp_save, csv_logger, Early_stop])

In [ ]:
myMdl = tf.keras.models.load_model('/content/drive/MyDrive/캡스톤디자인I/Save_Model/20240409_1modelsave/16-0.98459_best')

In [ ]:
trans_h = pd.read_csv('/content/drive/MyDrive/캡스톤디자인I/Save_Model/20240409_1modelsave/training.csv')

In [ ]:
#loss curve
plt.figure(figsize=(5,3))
plt.plot(trans_h['loss'][:], 'y', label='train loss')
plt.plot(trans_h['val_loss'][:], 'r', label='val loss')
plt.legend()

In [ ]:
#loss curve
plt.figure(figsize=(5,3))
plt.plot(trans_h['accuracy'][:], 'y', label='train accuracy')
plt.plot(trans_h['val_acurracy'][:], 'r', label='val accuracy')
plt.legend()